In [ ]:
import os
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import time

COIN_MARKET_CAP_API_KEY = os.getenv("COIN_MARKET_CAP_API_KEY")

In [7]:
class DataIngestor:
    def __init__(self):
        print("DataIngestor")
        print("DataIngestor initialized")

    def fetch_data(self):
        raise NotImplementedError("This method should be overridden by subclasses")

In [8]:
class CoinMarketCapIngestor(DataIngestor):
    def __init__(self, start=1, limit=5000, convert="USD"):
        super().__init__()
        self.headers = {
            "Accepts": "application/json",
            "X-CMC_PRO_API_KEY": COIN_MARKET_CAP_API_KEY,
        }
        self.session = Session()
        self.session.headers.update(self.headers)
        self.url = (
            "https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
        )
        self.parameters = {"start": start, "limit": limit, "convert": convert}

    def fetch_data(self):
        try:
            print("Fetching data from CoinMarketCap API")
            response = self.session.get(self.url, params=self.parameters)
            response.raise_for_status()  # Check if the request was successful
            data = response.json()  # Directly parse JSON response
            print(f"Data fetched successfully: {response}")
            return data
        except (ConnectionError, Timeout, TooManyRedirects, Exception) as e:
            print(f"An error occurred: {e}")
            return None

In [9]:
ingestor = CoinMarketCapIngestor()

DataIngestor
DataIngestor initialized


In [30]:
data = ingestor.fetch_data()

Fetching data from CoinMarketCap API
Data fetched successfully: <Response [200]>


In [12]:
data.keys()

dict_keys(['status', 'data'])

In [17]:
for key in data.keys():
    print(type(data[key]))

<class 'dict'>
<class 'list'>


In [58]:
len(data["data"])

10

In [57]:
data["data"][0].keys()

dict_keys(['id', 'name', 'symbol', 'slug', 'cmc_rank', 'num_market_pairs', 'circulating_supply', 'total_supply', 'max_supply', 'infinite_supply', 'last_updated', 'date_added', 'tags', 'platform', 'self_reported_circulating_supply', 'self_reported_market_cap', 'quote'])

In [26]:
pd.DataFrame(data["data"]).head()

,id,name,symbol,slug,cmc_rank,num_market_pairs,circulating_supply,total_supply,max_supply,infinite_supply,last_updated,date_added,tags,platform,self_reported_circulating_supply,self_reported_market_cap,quote
0,6815,3r3jkjygksg,urrlc86pk59,jer4ugvu3es,3818,9383,2450,8705,5186,None,2024-12-01T18:54:54.996Z,2024-12-01T18:54:54.996Z,"[mzuyaubu4q, qoqanbdttg, wzyxhbnwhos, jxxnzwaj...",None,None,None,"{'USD': {'price': 0.9316735000565917, 'volume_..."
1,395,61k303y92wl,q21fogpxyy,o94zc9xtfq,4609,2068,1280,3129,2714,None,2024-12-01T18:54:54.996Z,2024-12-01T18:54:54.996Z,"[7x49ln3cqb8, uf4mb51lqa, w1zzmxuxbce, ztcjmq2...",None,None,None,"{'USD': {'price': 0.39270123064507567, 'volume..."
2,7655,jxagndvl4e,3z8kft722uw,ujzqchvg0p,635,3948,1439,9856,3824,None,2024-12-01T18:54:54.996Z,2024-12-01T18:54:54.996Z,"[os5uc1kqfx, ag6y7ovk4hp, rzok077wv09, 9f6nyio...",None,None,None,"{'USD': {'price': 0.3937140643225703, 'volume_..."
3,8634,5ewozk5zxyc,noeq981pwnd,e7vdlnpgioo,1241,9348,1208,6810,1470,None,2024-12-01T18:54:54.996Z,2024-12-01T18:54:54.996Z,"[8zuy4vmmq2d, rvcfsgx8u5t, ltwf1tay7h, f2hyfex...",None,None,None,"{'USD': {'price': 0.34509711706880397, 'volume..."
4,1621,e1jgn8er30l,yh2y2o5bu8o,l1gsm2f570c,6372,6712,7682,4863,641,None,2024-12-01T18:54:54.996Z,2024-12-01T18:54:54.996Z,"[800yoljsqgj, 4zapoz19ste, vt3ttv524ej, 5i2z4f...",None,None,None,"{'USD': {'price': 0.8015683645864005, 'volume_..."


In [46]:
while True:
    data = ingestor.fetch_data()
    print(len(data["data"]))
    for i in range(len(data["data"])):
        print("".join(list(data["data"][i]["quote"]["USD"].values())))
    time.sleep(5)

Fetching data from CoinMarketCap API
Data fetched successfully: <Response [200]>
10


TypeError: sequence item 0: expected str instance, float found

In [74]:
s = ""
for i in list(data["data"][0]["quote"]["USD"].values()):
    s += rf"{ {i} } "

In [75]:
s

"{0.19984605551948165} {2153} {0.2787179068745387} {0.5606993435833614} {0.2139434805565712} {0.8195629606557195} {0.44857413595250994} {857} {0.11209255249181038} {'2024-12-01T19:51:45.101Z'} "